In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark
!pip install pyspark

In [ ]:
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [4]:
import findspark
findspark.init()

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import pyspark
import sys
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from uuid import uuid1
import time


kafka_topic_name  = "404-not-found"
# kafka_topic_name2 = "404-not-found-2"
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

In [6]:
import math

from pyspark.sql.functions import *

from pyspark import SparkContext

from pyspark.sql import SparkSession

In [7]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
import pandas as pd

In [55]:
sc = pyspark.SparkContext()
ssc = StreamingContext(sc,30)

In [56]:
# topic 1
kvs = KafkaUtils.createStream(ssc, kafka_bootstrap_servers, 'spark-streaming-consumer', {kafka_topic_name:1}) 
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {"metadata.broker.list": kafka_bootstrap_servers})
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'})

In [26]:
# # topic 2
# kvs2 = KafkaUtils.createStream(ssc, kafka_bootstrap_servers, 'spark-streaming-consumer', {kafka_topic_name2:1}) 
# kvs2 = KafkaUtils.createDirectStream(ssc, [kafka_topic_name2], {"metadata.broker.list": kafka_bootstrap_servers})
# kvs2 = KafkaUtils.createDirectStream(ssc, [kafka_topic_name2], {
#                         'bootstrap.servers':kafka_bootstrap_servers,
#                         'group.id':'test-group',
#                         'auto.offset.reset':'largest'})

In [ ]:
# kvs.pprint()
# # lines2 = kvs.map(lambda x: x[0],x[1])
# # lines2.pprint()
# lines = kvs.map(lambda x: x[1])
# lines.pprint()
# counts = lines.flatMap(lambda line: line.split(' ')).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b) 
# counts.pprint()
# ssc.start()
# # stream will run for 50 sec
# ssc.awaitTerminationOrTimeout(100)

In [9]:
# kvs.pprint()
# ssc.start()
# ssc.awaitTerminationOrTimeout(100)

In [8]:
# map1 = kvs.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])
# reduce=map1.reduceByKey(lambda x,y:x+y)
# tf=reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))
# map3=reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
# map4=map3.map(lambda x:(x[0],x[1][2]))
# reduce2=map4.reduceByKey(lambda x,y:x+y)
# idf=reduce2.map(lambda x: (x[0], math.log10(3/x[1])))
# rdd2=tf.join(idf)
# rdd2=rdd2.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1])))
# rdd2=rdd2.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
# rdd2.pprint()

# ssc.start()
# # stream will run for 50 sec
# ssc.awaitTerminationOrTimeout(100)

In [57]:
# data=[(1,'i love dogs'),(2,"i hate dogs and knitting"),(3,"knitting is my hobby and my passion")]
# lines=sc.parallelize(data)
# lines.collect()

# numLine = kvs2.flatMap(lambda x: x[1]).cast("int")

wind = kvs.window(30)

wind.count().pprint()

map1 = wind.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])

reduce = map1.reduceByKey(lambda x,y:x+y)

tf = reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))

map3 = reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))

# map3.collect()

map4 = map3.map(lambda x:(x[0],x[1][2]))

# map4.collect()

reduce2 = map4.reduceByKey(lambda x,y:x+y)

# reduce2.collect()

idf=reduce2.map(lambda x: (x[0],math.log10(347/x[1])))

# idf.collect()
###
rdd=tf.join(idf)

# rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1])))

rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))

# rdd.collect()
# rdd.pprint(2000)
# rdd.toDF(["DocumentId","Token","TF","IDF","TF-IDF"]).show()
###

############## importane word ######################
rdd2=rdd.map(lambda x: (x[0],(x[1],x[4])))

rdd2=rdd2.reduceByKey(lambda x,y: x if x[1] > y[1] else y)

important_w = rdd2.transform((lambda x: x.sortBy(lambda x:(-x[1][1]))))

important_w.pprint(400)

# rdd2 = rdd.transform((lambda x: x.sortBy(lambda x:(-x[4]))))
# important_w = rdd2.transform(lambda rdd:sc.parallelize(rdd.take(2000)))

# important_w = important_w.transform((lambda x:x.sortBy(lambda x:(int(x[0]))).filter(lambda x: x[4]>3)))

# important_w.pprint(2000)

In [58]:
ssc.start()
# stream will run for 120 sec
ssc.awaitTerminationOrTimeout(100)

-------------------------------------------
Time: 2021-09-24 17:43:30
-------------------------------------------
0

-------------------------------------------
Time: 2021-09-24 17:43:30
-------------------------------------------

-------------------------------------------
Time: 2021-09-24 17:44:00
-------------------------------------------
347

-------------------------------------------
Time: 2021-09-24 17:44:00
-------------------------------------------
('284', ('ronan', 14.097425795257841))
('311', ('plant', 13.56788638424038))
('95', ('curious', 13.435796874761355))
('8', ('dursley', 13.098805534125997))
('197', ('troll', 12.278143941616731))
('30', ('snake', 11.866620043436319))
('3', ('dursley', 11.788924980713396))
('82', ('goblin', 11.629616900777467))
('189', ('bludgers', 11.629616900777467))
('188', ('hoops', 11.196497395634461))
('290', ('firenze', 10.573069346443381))
('78', ('p', 10.573069346443381))
('289', ('firenze', 10.573069346443381))
('4', ('dursley', 10.479044

False

In [59]:
ssc.stop()
sc.stop()